# Job Dataset Exploration

In [13]:
import pandas as pd
import numpy as np
from lenskit.algorithms import Recommender
from lenskit.algorithms.user_knn import UserUser
from lenskit.algorithms import user_knn
from lenskit.algorithms import Recommender

In [14]:
jobs_data = pd.read_csv("dataset/splitjobs/jobs1.tsv", sep='\t', error_bad_lines=False)
jobs_df = jobs_data[["JobID", "Title"]]
jobs_df = jobs_df.rename(columns={"JobID": "item", "Title": "title"})

/var/folders/vt/47mxbcvj5rd7mmyk053h49m00000gn/T/ipykernel_78125/1825902803.py:1: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  jobs_data = pd.read_csv("dataset/splitjobs/jobs1.tsv", sep='\t', error_bad_lines=False)
b'Skipping line 122433: expected 11 fields, saw 12\n'
/var/folders/vt/47mxbcvj5rd7mmyk053h49m00000gn/T/ipykernel_78125/1825902803.py:1: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  jobs_data = pd.read_csv("dataset/splitjobs/jobs1.tsv", sep='\t', error_bad_lines=False)


In [15]:
# Block which concatenates popular_jobs estimation with the apps and assigng 
#   correct rating scores (1 if applied, 0 otherwise)

# read the data (about applications)
popular_data = pd.read_csv("dataset/popular_jobs.csv")
apps_df = pd.read_csv("dataset/apps.tsv", "\t")
apps_df = apps_df.loc[apps_df['Split'] == "Test"]
apps_df = apps_df.rename(columns={"UserID": "user", "JobID": "item"})
apps_df = apps_df[["user", "item"]]
apps_df.insert(2, "rating", np.full(apps_df.shape[0],1.0))  

data = popular_data.assign(JobIds=popular_data['JobIds'].str.split(' ')).explode('JobIds', ignore_index=True)
data.insert(2, "rating", np.full(data.shape[0],0.0))  
df_ui = data.rename(columns={"UserId": "user", "JobIds": "item"})


# concatenate, descending ratings, filter out duplicates
tmp = [df_ui, apps_df]
result = pd.concat(tmp).sort_values("rating", ascending=False)
result = result.drop_duplicates(subset=['user', 'item'])
result = result.sort_values('user', ascending=True).reset_index()
result = result.drop("index", axis=1)
result = result.dropna()
result["user"] = result["user"].astype(int)
result["item"] = result["item"].astype(int)
display(result.head(10))


/Users/kirillgugunishvili/opt/anaconda3/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3369: FutureWarning: In a future version of pandas all arguments of read_csv except for the argument 'filepath_or_buffer' will be keyword-only.
  exec(code_obj, self.user_global_ns, self.user_ns)


,user,item,rating
0,13,452170,0.0
1,13,511327,0.0
2,13,1033886,0.0
3,13,841119,0.0
4,13,361559,0.0
5,13,97494,0.0
6,13,638349,0.0
7,13,1066611,0.0
8,13,286627,0.0
9,13,585616,0.0


In [22]:
display(result.dtypes)
# We use the collaborative user algorithm UserUser, that use the nearest neighbors 
user_user = UserUser(10, min_nbrs=3)  # Minimum (3) and maximum (15) number of neighbors to consider
recsys = Recommender.adapt(user_user)
recsys.fit(result)

user        int64
item        int64
rating    float64
dtype: object

In [34]:
user_id = 18817 #2/2

selected_jobs = recsys.recommend(user_id, 100) # generate 10 recommendations for the selected user
# score - doesn't matter cause arbitrary value showing recommendation priority
# sometimes ipynb doesn't reset the variables -> gives stupid errors (solution shove everything into one kernel/ block)

selected_jobs = selected_jobs.join(jobs_df.set_index('item'), on='item')
display(selected_jobs.head(100))

# Show the real applications
apps_data = pd.read_csv("dataset/apps.tsv", sep='\t')
apps_data = apps_data.loc[apps_data['UserID'] == user_id]
apps_data = apps_data.rename(columns={"JobID": "item"})
display(apps_data.join(jobs_df.set_index('item'), on='item'))

# Idea:
#   - Take the test user and only his instances
#   - Use the test instances of other users as training instances
# Hit rate (Individual Rec. evaluation):
#   - Hopefully higher than 50%
#   - First 1000 users (which have recommendation) 
# Data for user studies:
#   - Roleplay as test users (i.e same users)
# Decoupled evaluation (Group Rec. evaluation):
#   - Note coupled because we don't have real data scores
# Assignment requirements:
#   - need evaluation of both individual and recommendation algorithms


,item,score,title
0,245220,0.948307,Apartment Lead Maintenance
1,796032,0.804134,Assistant Controller
2,298653,0.799571,Accounting Manager (Publicly Traded Organizati...
3,860385,0.798947,Senior Accountant
4,286222,0.781116,SAP Planning Analyst
...,...,...,...
70,33320,-0.201622,SQL Developer
71,612177,-0.201622,Financial Analyst
72,632902,-0.201622,Manufacturing Buyer
73,636386,-0.201622,Sales Representative / Account Manager / Cust...


,UserID,WindowID,Split,ApplicationDate,item,title
3435,18817,1,Test,2012-04-01 12:20:59.593,466017,Experienced Sales Professionals


In [18]:
# instead of recommender try to train predictor
user_user_2 = UserUser(15, min_nbrs=3)  # Minimum (3) and maximum (15) number of neighbors to consider
recsys = user_user_2.fit(result)

In [33]:
# predict based on previous code block
pred_user = 18817
pred_item = np.array([110292,466017,164195,168134])

output = recsys.predict_for_user(pred_user, pred_item)
display(output)

item
110292   -0.024685
466017         NaN
164195         NaN
168134         NaN
Name: prediction, dtype: float64